# ALGORITMOS BASADOS EN SERIES TEMPORALES

## A) Recopilación de Funciones

Se compilan los Algoritmos Probados

In [31]:
# LIBRERIAS NECESARIAS 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pyodbc
from dotenv import dotenv_values

# Mostrar el DataFrame resultante
import ace_tools_open as tools

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category= FutureWarning)

In [ ]:
# FUNCIONES

###----------------------------------------------------------------
#     DATOS
###----------------------------------------------------------------
def Open_Connection():
    secrets = dotenv_values(".env")   # Connection String from .env
    conn_str = f'DRIVER={secrets["DRIVER2"]};SERVER={secrets["SERVIDOR2"]};PORT={secrets["PUERTO2"]};DATABASE={secrets["BASE2"]};UID={secrets["USUARIO2"]};PWD={secrets["CONTRASENA2"]}'
    # print (conn_str) 
    try:    
        conn = pyodbc.connect(conn_str)
        return conn
    except:
        print('Error en la Conexión')
        return None
    
def Close_Connection(conn): 
    conn.close()
    return True

def generar_datos(id_proveedor, etiqueta):
    #  Intento recuperar datos cacheados
    try:
        data = pd.read_csv(f'data/{etiqueta}.csv')
        data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
        data['Sucursal']= data['Sucursal'].astype(int)
        data['Fecha']= pd.to_datetime(data['Fecha'])

        articulos = pd.read_csv(f'data/{etiqueta}_articulos.csv')
        #articulos.head()
        print(f"-> Datos Recuperados del CACHE: {id_proveedor}, Label: {etiqueta}")
        return data, articulos
    except:     
        print(f"-> Generando datos para ID: {id_proveedor}, Label: {etiqueta}")
        # Configuración de conexión
        conn = Open_Connection()
        
        # FILTRA solo PRODUCTOS HABILITADOS y Traer datos de STOCK y PENDIENTES desde PRODUCCIÓN
        query = f"""
        SELECT  A.[C_PROVEEDOR_PRIMARIO]
            ,S.[C_ARTICULO]
            ,S.[C_SUCU_EMPR]
            ,S.[Q_FACTOR_VENTA_ESP]
            ,S.[Q_FACTOR_VTA_SUCU]
            ,S.[M_OFERTA_SUCU]
            ,S.[M_HABILITADO_SUCU]
            ,A.M_BAJA
            ,S.[Q_VTA_DIA_ANT]
            ,S.[Q_VTA_ACUM]
            ,S.[Q_ULT_ING_STOCK]
            ,S.[Q_STOCK_A_ULT_ING]
            ,S.[Q_15DIASVTA_A_ULT_ING_STOCK]
            ,S.[Q_30DIASVTA_A_ULT_ING_STOCK]
            ,S.[Q_BULTOS_PENDIENTE_OC]
            ,S.[Q_PESO_PENDIENTE_OC]
            ,S.[Q_UNID_PESO_PEND_RECEP_TRANSF]
            ,S.[Q_UNID_PESO_VTA_MES_ACTUAL]
            ,S.[F_ULTIMA_VTA]
            ,S.[Q_VTA_ULTIMOS_15DIAS]
            ,S.[Q_VTA_ULTIMOS_30DIAS]
            ,S.[Q_TRANSF_PEND]
            ,S.[Q_TRANSF_EN_PREP]
            ,S.[M_FOLDER]
            ,S.[M_ALTA_RENTABILIDAD]
            ,S.[Lugar_Abastecimiento]
            ,S.[M_COSTO_LOGISTICO]
        
        FROM [DIARCOP001].[DiarcoP].[dbo].[T051_ARTICULOS_SUCURSAL] S
        LEFT JOIN [DIARCOP001].[DiarcoP].[dbo].[T050_ARTICULOS] A
            ON A.[C_ARTICULO] = S.[C_ARTICULO]

        WHERE S.[M_HABILITADO_SUCU] = 'S' -- Permitido Reponer
            AND A.M_BAJA = 'N'  -- Activo en Maestro Artículos
            AND A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} -- Solo del Proveedor
        ;
        """
        # Ejecutar la consulta SQL
        articulos = pd.read_sql(query, conn)
        
        # Consulta SQL para obtener las ventas de un proveedor específico   
        # Reemplazar {proveedor} en la consulta con el ID de la tienda actual
        query = f"""
        SELECT V.[F_VENTA] as Fecha
            ,V.[C_ARTICULO] as Codigo_Articulo
            ,V.[C_SUCU_EMPR] as Sucursal
            ,V.[I_PRECIO_VENTA] as Precio
            ,V.[I_PRECIO_COSTO] as Costo
            ,V.[Q_UNIDADES_VENDIDAS] as Unidades
            ,V.[C_FAMILIA] as Familia
            ,A.[C_RUBRO] as Rubro
            ,A.[C_SUBRUBRO_1] as SubRubro
            ,LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(A.N_ARTICULO, CHAR(9), ''), CHAR(13), ''), CHAR(10), ''))) as Nombre_Articulo
            ,A.[C_CLASIFICACION_COMPRA] as Clasificacion
        FROM [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO] V
        LEFT JOIN [DCO-DBCORE-P02].[DiarcoEst].[dbo].[T050_ARTICULOS] A 
            ON V.C_ARTICULO = A.C_ARTICULO
        WHERE A.[C_PROVEEDOR_PRIMARIO] = {id_proveedor} AND V.F_VENTA >= '20210101' AND A.M_BAJA ='N'
        ORDER BY V.F_VENTA ;
        """

        # Ejecutar la consulta SQL
        demanda = pd.read_sql(query, conn)
        
        # UNIR Y FILTRAR solo la demanda de los Hartículos VALIDOS.
        # Realizar la unión (merge) de los DataFrames por las claves especificadas
        data = pd.merge(
            articulos,  # DataFrame de artículos
            demanda,    # DataFrame de demanda
            left_on=['C_ARTICULO', 'C_SUCU_EMPR'],  # Claves en 'articulos'
            right_on=['Codigo_Articulo', 'Sucursal'],  # Claves en 'demanda'
            how='inner'  # Solo traer los productos que están en 'articulos'
        )
            
        # Guardar los resultados en un archivo CSV con el nombre del Proveedor
        file_path = f'data/{etiqueta}_FULL.csv'
        data.to_csv(file_path, index=False, encoding='utf-8')
        print(f"---> Datos de FULL guardados: {file_path}")
        
        file_path = f'data/{etiqueta}_articulos.csv'
        articulos.to_csv(file_path, index=False, encoding='utf-8')
        
        print(f"---> Datos de Artículos guardados: {file_path}")
        
        # Eliminar Columnas Innecesarias
        data = data[['Fecha', 'Codigo_Articulo', 'Sucursal', 'Unidades']]
        data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
        data['Sucursal']= data['Sucursal'].astype(int)
        
        # Guardar los resultados en un archivo CSV con el nombre del Proveedor
        file_path = f'data/{etiqueta}.csv'
        data.to_csv(file_path, index=False, encoding='utf-8')
        
        # Cerrar la conexión después de la iteración
        Close_Connection(conn)
        return data, articulos
###----------------------------------------------------------------
#     ALGORITMOS
###----------------------------------------------------------------
# ALGO_01 Promedio de Ventas Ponderado  
def Calcular_Demanda_ALGO_01(df, id_proveedor, etiqueta, period_length, current_date, factor_last, factor_previous, factor_year):
    print('Dentro del Calcular_Demanda_ALGO_01')
    print(f'FORECATS control: {id_proveedor} - {etiqueta} - ventana: {period_length} - factores: {factor_last} - {factor_previous} - {factor_year}')
    # Definir rangos de fechas para cada período
    last_period_start = current_date - pd.Timedelta(days=period_length - 1)
    last_period_end = current_date

    previous_period_start = current_date - pd.Timedelta(days=2 * period_length - 1)
    previous_period_end = current_date - pd.Timedelta(days=period_length)

    same_period_last_year_start = current_date - pd.DateOffset(years=1) - pd.Timedelta(days=period_length - 1)
    same_period_last_year_end = current_date - pd.DateOffset(years=1)

    # Filtrar los datos para cada uno de los períodos
    df_last = df[(df['Fecha'] >= last_period_start) & (df['Fecha'] <= last_period_end)]
    df_previous = df[(df['Fecha'] >= previous_period_start) & (df['Fecha'] <= previous_period_end)]
    df_same_year = df[(df['Fecha'] >= same_period_last_year_start) & (df['Fecha'] <= same_period_last_year_end)]

    # Agregar las ventas (unidades) por artículo y sucursal para cada período
    sales_last = df_last.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                        .sum().reset_index().rename(columns={'Unidades': 'ventas_last'})
    sales_previous = df_previous.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                .sum().reset_index().rename(columns={'Unidades': 'ventas_previous'})
    sales_same_year = df_same_year.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                .sum().reset_index().rename(columns={'Unidades': 'ventas_same_year'})

    # Unir la información de los tres períodos
    forecast_df = pd.merge(sales_last, sales_previous, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    forecast_df = pd.merge(forecast_df, sales_same_year, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    forecast_df.fillna(0, inplace=True)

    # Calcular la demanda estimada como el promedio de las ventas de los tres períodos
    forecast_df['Forecast'] = (forecast_df['ventas_last'] * factor_last +
                               forecast_df['ventas_previous'] * factor_previous +
                               forecast_df['ventas_same_year'] * factor_year) / (factor_year + factor_last + factor_previous)

    # Redondear la predicción al entero más cercano
    forecast_df['Forecast'] = forecast_df['Forecast'].round().astype(int)
    forecast_df['Average'] = round(forecast_df['Forecast'] /period_length ,3)
    # Borrar Columnas Innecesarias
    forecast_df.drop(columns=['ventas_last', 'ventas_previous', 'ventas_same_year'], inplace=True)    

    return forecast_df

def Calcular_Demanda_Extendida_ALGO_01(df, id_proveedor, etiqueta, period_length, current_date, factor_last, factor_previous, factor_year):
    # Definir rangos de fechas para cada período
    last_period_start = current_date - pd.Timedelta(days=period_length - 1)
    last_period_end = current_date

    previous_period_start = current_date - pd.Timedelta(days=2 * period_length - 1)
    previous_period_end = current_date - pd.Timedelta(days=period_length)

    same_period_last_year_start = current_date - pd.DateOffset(years=1) - pd.Timedelta(days=period_length - 1)
    same_period_last_year_end = current_date - pd.DateOffset(years=1)

    # Filtrar los datos para cada uno de los períodos
    df_last = df[(df['Fecha'] >= last_period_start) & (df['Fecha'] <= last_period_end)]
    df_previous = df[(df['Fecha'] >= previous_period_start) & (df['Fecha'] <= previous_period_end)]
    df_same_year = df[(df['Fecha'] >= same_period_last_year_start) & (df['Fecha'] <= same_period_last_year_end)]

    # Agregar las ventas (unidades) por artículo y sucursal para cada período mas todos los datos anteriores.
    sales_last = df_last.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                        .sum().reset_index().rename(columns={'Unidades': 'ventas_last'})
    sales_previous = df_previous.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                .sum().reset_index().rename(columns={'Unidades': 'ventas_previous'})
    sales_same_year = df_same_year.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                .sum().reset_index().rename(columns={'Unidades': 'ventas_same_year'})

    # Unir la información de los tres períodos
    validacion_df = pd.merge(sales_last, sales_previous, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    validacion_df = pd.merge(validacion_df, sales_same_year, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    validacion_df.fillna(0, inplace=True)

    # Calcular la demanda estimada como el promedio de las ventas de los tres períodos
    validacion_df['Forecast'] = (validacion_df['ventas_last'] * factor_last +
                               validacion_df['ventas_previous'] * factor_previous +
                               validacion_df['ventas_same_year'] * factor_year) / (factor_year + factor_last + factor_previous)
    validacion_df['Average'] = round(validacion_df['Forecast'] /period_length ,3)
    
    # Redondear la predicción al entero más cercano
    validacion_df['Forecast'] = validacion_df['Forecast'].round(2)

    return validacion_df

def Calcular_Demanda_ALGO_05(df_prv,forecast_window,id_proveedor, etiqueta):
    # Lista para almacenar los resultados del pronóstico
    resultados = []

    # Agrupar los datos por 'Codigo_Articulo' y 'Sucursal'
    for (codigo, sucursal), grupo in df_prv.groupby(['Codigo_Articulo', 'Sucursal']):
        # Establecer 'Fecha' como índice y ordenar los datos
        grupo = grupo.set_index('Fecha').sort_index()
        
        # Resamplear a diario sumando las ventas
        ventas_diarias = grupo['Unidades'].resample('D').sum().fillna(0)
        
        # Seleccionar un periodo reciente para calcular la media; por ejemplo, los últimos 30 días
        # Si hay menos de 30 días de datos, se utiliza el periodo disponible
        ventas_recientes = ventas_diarias[-30:]
        media_diaria = ventas_recientes.mean()
        
        # Pronosticar la demanda para el periodo de reposición
        pronostico = media_diaria * forecast_window
        
        resultados.append({
            'Codigo_Articulo': codigo,
            'Sucursal': sucursal,
            'Forecast': round(pronostico, 2),
            'Average': round(media_diaria, 3)
        })

    # Crear el DataFrame de pronósticos
    df_pronostico = pd.DataFrame(resultados)

    return df_pronostico


def Calcular_Demanda_Extendida_ALGO_05(df_prv,forecast_window,id_proveedor, etiqueta):
    max_date = df_prv['Fecha'].max()
    cutoff_date = max_date - timedelta(days=forecast_window)
    resultados_validacion = []

    # Agrupar los datos por 'Codigo_Articulo' y 'Sucursal'
    for (codigo, sucursal), grupo in df_prv.groupby(['Codigo_Articulo', 'Sucursal']):
        # Establecer 'Fecha' como índice y ordenar cronológicamente
        grupo = grupo.set_index('Fecha').sort_index()
        
        # Datos de entrenamiento: hasta la fecha de corte
        datos_entrenamiento = grupo.loc[:cutoff_date]
        
        # Verificar que existan suficientes datos para calcular el promedio (por ejemplo, al menos 30 días)
        if len(datos_entrenamiento) < 30:
            continue  # O vemos como manejarlo de otra forma
        
        # Resamplear a ventas diarias en el conjunto de entrenamiento
        ventas_diarias_entrenamiento = datos_entrenamiento['Unidades'].resample('D').sum().fillna(0)
        
        # Calcular la media diaria de los últimos 30 días del periodo de entrenamiento
        ventas_recientes = ventas_diarias_entrenamiento[-30:]
        media_diaria = ventas_recientes.mean()
        
        # Calcular el pronóstico para la ventana definida
        forecast = media_diaria * forecast_window
        
        # Definir el periodo de validación: desde el día siguiente a la fecha de corte hasta completar la ventana
        inicio_validacion = cutoff_date + timedelta(days=1)
        fin_validacion = cutoff_date + timedelta(days=forecast_window)
        
        # Extraer y resumir las ventas reales en el periodo de validación
        ventas_validacion = grupo.loc[inicio_validacion:fin_validacion]['Unidades'].resample('D').sum().fillna(0)
        ventas_reales = ventas_validacion.sum()
        
        # Calcular medidas de error
        error_absoluto = abs(forecast - ventas_reales)
        error_porcentual = (error_absoluto / ventas_reales * 100) if ventas_reales != 0 else None
        
        resultados_validacion.append({
            'Codigo_Articulo': codigo,
            'Sucursal': sucursal,
            'Forecast': round(forecast, 2),
            'Ventas_Reales': round(ventas_reales, 2),
            'Error_Absoluto': round(error_absoluto, 2),
            'Error_Porcentual': round(error_porcentual, 2) if error_porcentual is not None else None
        })

    # Crear un DataFrame con los resultados de validación
    df_validacion = pd.DataFrame(resultados_validacion)

    return df_validacion


def Exportar_Pronostico(df_forecast, etiqueta, algoritmo):
    df_forecast['Codigo_Articulo']= df_forecast['Codigo_Articulo'].astype(int)
    df_forecast['Sucursal']= df_forecast['Sucursal'].astype(int)
    
    # tools.display_dataframe_to_user(name="SET de Datos del Proveedor", dataframe=df_forecast)
    # df_forecast.info()
    print(f'-> ** Pronostico Exportado: data/{etiqueta}_Pronostico_{algoritmo}.csv **')
    df_forecast.to_csv(f'data/{etiqueta}_Pronostico_{algoritmo}.csv', index=False)
    return


def Procesar_ALGO_05(data, proveedor, etiqueta, ventana, fecha):
    df_forecast = Calcular_Demanda_ALGO_05(data, ventana, proveedor, etiqueta)    # Exportar el resultado a un CSV para su posterior procesamiento
    df_forecast['Codigo_Articulo']= df_forecast['Codigo_Articulo'].astype(int)
    df_forecast['Sucursal']= df_forecast['Sucursal'].astype(int)
    df_forecast.to_csv(f'data/{etiqueta}_ALGO_05_Solicitudes_Compra.csv', index=False)

    df_validacion = Calcular_Demanda_Extendida_ALGO_05(data, ventana, proveedor, etiqueta)
    df_validacion['Codigo_Articulo']= df_validacion['Codigo_Articulo'].astype(int)
    df_validacion['Sucursal']= df_validacion['Sucursal'].astype(int)
    df_validacion.to_csv(f'data/{etiqueta}_ALGO_05_Datos_Validacion.csv', index=False)
    print(f'-> ** Validación Exportada: {etiqueta}_ALGO_05_Datos_Validacion.csv *** : ventana: {ventana}  - {fecha}')
    
    Exportar_Pronostico(df_forecast, etiqueta, 'ALGO_05')  # Impactar Datos en la Interface   
    return

def Procesar_ALGO_01(data, proveedor, etiqueta, ventana, fecha, factor_last=None, factor_previous=None, factor_year=None):    
    # Asignar valores por defecto si los factores no están definidos
    factor_last = 77 if factor_last is None else int(factor_last)
    factor_previous = 22 if factor_previous is None else int(factor_previous)
    factor_year = 11 if factor_year is None else int(factor_year)

    print(f'--> ALGO_01 ventana {ventana} - Peso de los Factores Utilizados: último: {factor_last} previo: {factor_previous} año anterior: {factor_year}')
        
    df_forecast = Calcular_Demanda_ALGO_01(data, proveedor, etiqueta, ventana, fecha, factor_last, factor_previous, factor_year)
    df_forecast['Codigo_Articulo']= df_forecast['Codigo_Articulo'].astype(int)
    df_forecast['Sucursal']= df_forecast['Sucursal'].astype(int)
    df_forecast.to_csv(f'data/{etiqueta}_ALGO_01_Solicitudes_Compra.csv', index=False)   # Exportar el resultado a un CSV para su posterior procesamiento
    
    df_validacion = Calcular_Demanda_Extendida_ALGO_01(data, proveedor, etiqueta, ventana, fecha, factor_last, factor_previous, factor_year)
    df_validacion['Codigo_Articulo']= df_validacion['Codigo_Articulo'].astype(int)
    df_validacion['Sucursal']= df_validacion['Sucursal'].astype(int)
    df_validacion.to_csv(f'data/{etiqueta}_ALGO_01_Datos_Validacion.csv', index=False)
    print(f'-> ** Validación Exportada: {etiqueta}_ALGO_01_Datos_Validacion.csv *** : ventana: {ventana}  - {fecha}')
    
    Exportar_Pronostico(df_forecast, etiqueta, 'ALGO_01')  # Impactar Datos en la Interface        
    return



In [33]:
# RUTINA PRINCIPAL para obtener el pronóstico
def get_forecast( id_proveedor, lbl_proveedor, period_lengh=30, algorithm='basic', f1=None, f2=None, f3=None, current_date=None ):
    """
    Genera la predicción de demanda según el algoritmo seleccionado.

    Parámetros:
    - id_proveedor: ID del proveedor.
    - lbl_proveedor: Etiqueta del proveedor.
    - period_lengh: Número de días del período a analizar (por defecto 30).
    - algorithm: Algoritmo a utilizar.
    - current_date: Fecha de referencia; si es None, se toma la fecha máxima de los datos.
    - factores de ponderación: F1, F2, F3  (No importa en que unidades estén, luego los hace relativos al total del peso)

    Retorna:
    - Un DataFrame con las predicciones.
    """
    
    print('Dentro del get_forecast')
    print(f'FORECAST control: {id_proveedor} - {lbl_proveedor} - ventana: {period_lengh} - {algorithm} factores: {f1} - {f2} - {f3}')
    # Generar los datos de entrada
    data, articulos = generar_datos(id_proveedor, lbl_proveedor)

        # Determinar la fecha base
    if current_date is None:
        current_date = data['Fecha'].max()  # Se toma la última fecha en los datos
    else:
        current_date = pd.to_datetime(current_date)  # Se asegura que sea un objeto datetime

    print(f'Fecha actual {current_date}')
    

    # Selección del algoritmo de predicción
    match algorithm:
        case 'ALGO_01':
            return Procesar_ALGO_01(data, id_proveedor, lbl_proveedor, period_lengh, current_date, f1, f2, f3)
        case 'ALGO_05':
            return Procesar_ALGO_05(data, id_proveedor, lbl_proveedor, period_lengh, current_date)
        case _:
            raise ValueError(f"Error: El algoritmo '{algorithm}' no está implementado.")



In [41]:
#EJECUCIÓN MASIVA x LISTA

proveedores = [
    # {"id": 20, "nombre": "MOLINOS RIO DE LA PLATA", "label": "20_MOLINOS", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 20, "nombre": "MOLINOS RIO DE LA PLATA", "label": "20_MOLINOS", "ventana": 30, "algoritmo" : "ALGO_01", "f1": 90, "f2": 10, "f3": 20},
    # {"id": 25, "nombre": "CAFES LA VIRGINIA S.A.", "label": "25_LA_VIRGINIA", "ventana": 30, "algoritmo"  :  "ALGO_05"},
    # {"id": 25, "nombre": "CAFES LA VIRGINIA S.A.", "label": "25_LA_VIRGINIA", "ventana": 30, "algoritmo"  :   "ALGO_01", "f1": 80, "f2": 10, "f3": 20},
    # {"id": 62, "nombre": "ARCOR","label":"62_ARCOR", "label": "62_ARCOR", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 98, "nombre": "FRATELLI BRANCA DESTILERIAS S.A.", "label": "98_FRATELLI_BRANCA", "ventana": 30, "algoritmo" : "ALGO_01", "f1": 70, "f2": 10, "f3": 20},
    # {"id": 98, "nombre": "FRATELLI BRANCA DESTILERIAS S.A.", "label": "98_FRATELLI_BRANCA", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 140, "nombre": "UNILEVER DE ARGENTINA S.A.", "label": "140_UNILEVER", "ventana": 30, "algoritmo" : "ALGO_05"},
    {"id": 189, "nombre": "BODEGAS Y VIÑEDOS LOPEZ S.A.I.C.", "label": "189_BODEGAS_LOPEZ", "ventana": 30, "algoritmo" : "ALGO_05"},
    {"id": 189, "nombre": "BODEGAS Y VIÑEDOS LOPEZ S.A.I.C.", "label": "189_BODEGAS_LOPEZ", "ventana": 30, "algoritmo" : "ALGO_01", "f1": 100, "f2": 10, "f3": 10},
    # {"id": 1465, "nombre": "QUICKFOOD S.A.", "label":"1465_QUICKFOOD", "ventana": 30, "algoritmo" : "ALGO_05"},
    # {"id": 327, "nombre": "PALADINI S.A.", "label":"327_PALADINI", "ventana": 30, "algoritmo" : "ALGO_05"}
]

for proveedor in proveedores:
    f1 = proveedor.get("f1", None)  # Si no está en el diccionario, devuelve None
    f2 = proveedor.get("f2", None)
    f3 = proveedor.get("f3", None)
    
    print(f'Procesando: {proveedor["id"]} - {proveedor["label"]} - ventana: {proveedor["ventana"]} - {proveedor["algoritmo"]} factores: {f1} - {f2} - {f3}')
    
    #get_forecast( proveedor["id"], proveedor["label"], proveedor["ventana"], proveedor["algoritmo"], proveedor["f1"], proveedor["f2"], proveedor["f3"])
    get_forecast(proveedor["id"], proveedor["label"], proveedor["ventana"], proveedor["algoritmo"], f1, f2, f3)
    
    print('------------------------------------------------------------------')
    
    
    


Procesando: 189 - 189_BODEGAS_LOPEZ - ventana: 30 - ALGO_05 factores: None - None - None
Dentro del get_forecast
FORECAST control: 189 - 189_BODEGAS_LOPEZ - ventana: 30 - ALGO_05 factores: None - None - None
-> Generando datos para ID: 189, Label: 189_BODEGAS_LOPEZ
---> Datos de FULL guardados: data/189_BODEGAS_LOPEZ_FULL.csv
---> Datos de Artículos guardados: data/189_BODEGAS_LOPEZ_articulos.csv
Fecha actual 2025-02-23 00:00:00
-> ** Validación Exportada: 189_BODEGAS_LOPEZ_ALGO_05_Datos_Validacion.csv *** : ventana: 30  - 2025-02-23 00:00:00
-> ** Pronostico Exportado: data/189_BODEGAS_LOPEZ_Pronostico_ALGO_05.csv **
------------------------------------------------------------------
Procesando: 189 - 189_BODEGAS_LOPEZ - ventana: 30 - ALGO_01 factores: 100 - 10 - 10
Dentro del get_forecast
FORECAST control: 189 - 189_BODEGAS_LOPEZ - ventana: 30 - ALGO_01 factores: 100 - 10 - 10
-> Datos Recuperados del CACHE: 189, Label: 189_BODEGAS_LOPEZ
Fecha actual 2025-02-23 00:00:00
--> ALGO_01 ve

In [37]:
etiqueta ='189_BODEGAS_LOPEZ'
data = pd.read_csv(f'data/{etiqueta}.csv')
data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
data['Sucursal']= data['Sucursal'].astype(int)
data['Fecha']= pd.to_datetime(data['Fecha'])

# import ace_tools_open as tools
tools.display_dataframe_to_user(name="SET de Datos del Proveedor", dataframe=data)

SET de Datos del Proveedor


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16678 entries, 0 to 16677
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fecha            16678 non-null  datetime64[ns]
 1   Codigo_Articulo  16678 non-null  int32         
 2   Sucursal         16678 non-null  int32         
 3   Unidades         16678 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(2)
memory usage: 391.0 KB
